### Устанавливаем и запускаем Vertica

In [ ]:
pip install vertica-python

In [2]:
import os

In [20]:
os.system('docker run -d -p 5433:5433 jbfavre/vertica:latest')

0

### Подключаемся к БД

In [83]:
import vertica_python
import os

In [84]:
connection_info = {
    'host': 'localhost',
    'port': 5433,
    'user': 'dbadmin',
    'password': '',
    'database': 'docker',
    'autocommit': True,
} 

### Проводим Миграции

In [ ]:
connection =  vertica_python.connect(**connection_info)
cursor = connection.cursor()

In [106]:

cursor.execute("""CREATE TABLE movie_view (
    id IDENTITY, user_uuid VARCHAR(256), movie_uuid VARCHAR(256), movie_progress VARCHAR(256), movie_length VARCHAR(256), event_time VARCHAR(256));
""")

### Заполняем БД на 1 000 000 записей

In [89]:
import time
import pandas
from pydantic import parse_obj_as, BaseModel
import datetime
from typing import List
import random
import uuid

In [90]:
movie_uuids = [str(uuid.uuid4()) for _ in range(1000)]
user_uuids = [str(uuid.uuid4()) for _ in range(2000)]

In [115]:
length = random.randint(600, 90000)
movies_gen = ((
    random.choice(user_uuids), random.choice(movie_uuids), str(random.randint(0, length)), str(length), str(datetime.datetime.now())) for _ in range(10000000)
)


In [ ]:
%%time
step = 10000
for i in range(0, 100):
    movies = [next(movies_gen) for _ in range(step)]
    cursor.executemany("""INSERT INTO movie_view (
    user_uuid, movie_uuid, movie_progress, movie_length, event_time) VALUES (?, ?, ?, ?, ?)""", movies, use_prepared_statements=True) 
